In [290]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Reshape, Input

In [291]:
# generate data
if 0:
    for i in range(6):
        A = np.random.random((10,))
        np.save('data/data{}.npy'.format(i), A)
print(A, A.shape)

[0.96845043 0.64812131 0.27250078 0.96509615 0.28636066 0.95594625
 0.20458066 0.63759591 0.4749517  0.77248394] (10,)


In [292]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, n_classes, shuffle):
        'Initialization'
        self.list_IDs = list_IDs
        self.labels = labels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / 1))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index:(index+1)]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        ID = list_IDs_temp[0]
        X = np.load('data/' + ID + '.npy')
        X = X[np.newaxis, :]
        
        y = np.zeros(self.n_classes)
        y[self.labels[ID]] = 1
        y = y[np.newaxis, :]
        
        return X, y

In [293]:
# Parameters
n_classes = 3
shuffle = True

# Datasets
partition = {'train': ['data0', 'data1', 'data2'], 'validation': ['data3', 'data4', 'data5']}
labels = {'data0': 0, 'data1': 1, 'data2': 2, 'data3': 1, 'data4': 2, 'data5': 0}

In [294]:
training_generator = DataGenerator(partition['train'], labels, n_classes, shuffle)
validation_generator = DataGenerator(partition['validation'], labels, n_classes, shuffle)

for i in range(len(training_generator)):
    item = training_generator.__getitem__(i)
    print(item, item[0].shape, item[1].shape)

(array([[0.94569562, 0.07265908, 0.44589503, 0.5849801 , 0.46483884,
        0.52023664, 0.18430294, 0.16623775, 0.70766679, 0.63499653]]), array([[1., 0., 0.]])) (1, 10) (1, 3)
(array([[0.84448407, 0.92908541, 0.90640278, 0.46929053, 0.72223718,
        0.38170618, 0.64859279, 0.27141775, 0.57673044, 0.40825563]]), array([[0., 0., 1.]])) (1, 10) (1, 3)
(array([[0.03280069, 0.46863122, 0.92635335, 0.51834472, 0.57592493,
        0.49837597, 0.9402426 , 0.62870501, 0.5397904 , 0.64768882]]), array([[0., 1., 0.]])) (1, 10) (1, 3)


In [295]:
model = Sequential([
    Dense(10, input_shape=(10,)),
    Activation('relu'),
    Dense(5),
    Activation('softmax'),
    Dense(3),
])
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), \
    metrics=[tf.keras.metrics.CategoricalCrossentropy()])
model.summary()

Model: "sequential_57"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_164 (Dense)            (None, 10)                110       
_________________________________________________________________
activation_120 (Activation)  (None, 10)                0         
_________________________________________________________________
dense_165 (Dense)            (None, 5)                 55        
_________________________________________________________________
activation_121 (Activation)  (None, 5)                 0         
_________________________________________________________________
dense_166 (Dense)            (None, 3)                 18        
Total params: 183
Trainable params: 183
Non-trainable params: 0
_________________________________________________________________


In [296]:
# Train model on dataset
history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5)

Epoch 1/5
3/3 [==============================] - 0s 60ms/step - loss: 1.4613 - categorical_crossentropy: 1.4613 - val_loss: 5.9058 - val_categorical_crossentropy: 5.9058
Epoch 2/5
3/3 [==============================] - 0s 52ms/step - loss: 1.2193 - categorical_crossentropy: 1.2193 - val_loss: 5.9161 - val_categorical_crossentropy: 5.9161
Epoch 3/5
3/3 [==============================] - 0s 50ms/step - loss: 1.0650 - categorical_crossentropy: 1.0650 - val_loss: 5.9179 - val_categorical_crossentropy: 5.9179
Epoch 4/5
3/3 [==============================] - 0s 50ms/step - loss: 0.9852 - categorical_crossentropy: 0.9852 - val_loss: 5.9218 - val_categorical_crossentropy: 5.9218
Epoch 5/5
3/3 [==============================] - 0s 50ms/step - loss: 0.9363 - categorical_crossentropy: 0.9363 - val_loss: 5.9307 - val_categorical_crossentropy: 5.9307
